# Barbuilder
## An Interactive Cocktail Ingredient Recommendation System

### Set Up System

In [104]:
# imports

import base64
from bitarray import bitarray
import csv
import io
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import plotly.express as px
import urllib
from wordcloud import WordCloud

In [105]:
# locations for file imports

processed_data = Path.cwd() / 'processed' / 'cosine_matrix.pickle'
ingredient_data = Path.cwd() / 'processed' / 'ingredient_dict.pickle'
recipe_data = Path.cwd() / 'processed' / 'recipe_dict.pickle'

In [106]:
# load Data (Generate data if not found, can take several minutes on first run to download raw files)

# check if data already generated. If not, run data_processing.ipynb.
try:
    open(processed_data, mode='r', encoding='utf-8')
    open(ingredient_data, mode='r', encoding='utf-8')
except FileNotFoundError as err:
    %run ./data_processing.ipynb

#  load the data files
with open(processed_data,'rb') as modelFile:
     df_sims = pickle.load(modelFile)
with open(ingredient_data,'rb') as modelFile:
     df_ingredients = pickle.load(modelFile)
with open(recipe_data,'rb') as modelFile:
     recipe_dict = pickle.load(modelFile)

In [107]:
# prepare data for different uses

# clean up recipe_dict (remove empty strings)  {recipe: [ingredients]}
for r in recipe_dict.keys():
    filter_object = filter(lambda x: x != "", recipe_dict[r])
    without_empty_strings = list(filter_object)
    recipe_dict[r] = without_empty_strings

# calculate and sort a dataframe of recipes / ingredient
recipes_per_ingredient = df_ingredients.sum(axis=1).sort_values(ascending=False).copy()

# dictionary of all recipes in each ingredient {ingredient: [recipes]}
ingredient_dict = {} 
for ingredient in df_ingredients.index:
    ingredient_dict[ingredient] = df_ingredients.loc[ingredient].index[df_ingredients.loc[ingredient] > 0]

#### Generate Graphics

In [146]:
# bar graph of recipes / ingredient
filtered_ingred = recipes_per_ingredient >= 10
x_val = recipes_per_ingredient.index[filtered_ingred] 
y_val = recipes_per_ingredient[filtered_ingred] 
graph_recipes_per_ingredient = px.bar(x=x_val, y=y_val, width=1000, range_x=[-1,20], title='Recipes / Ingredient (Minimum 10)', 
            labels={'x':'Ingredient', 'y':'Total Recipes'})
# graph_recipes_per_ingredient.show()

In [147]:
# heatmap with hover
 #click/drag to zoom. double click to zoom out
heatmap_ingredients = px.imshow(df_sims, color_continuous_scale='electric', aspect="equal", height=1000)
# heatmap_ingredients.show()

In [148]:
# wordcloud of ingredient frequencies
wordcloud = WordCloud(width = 1000, height = 500).generate_from_frequencies(recipes_per_ingredient)

plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.axis("off")
buf = io.BytesIO()
plt.savefig(buf, format='png')
buf.seek(0)
string = base64.b64encode(buf.read())

uri = 'data:image/png;base64,' + urllib.parse.quote(string)
plt.close()
wordcloud_ingredients_html = '<img src = "%s"/>' % uri

# Data Analysis (Descriptive Methods)

●	Three visualizations (images). Static images are permissible. 
●	A Descriptive method = anything that describes the data. 
○	Images can double count as your visualization and descriptive method.
○	Ex. mean, median, bar plot, scatterplot, k-means clustering, etc. 
●	A Non-descriptive method = anything that infers from the data, i.e., makes predictions or prescriptions. 
○	Ex. classification models, regression, image recognition, etc. 
●	An application of “machine learning” in the non-descriptive OR descriptive method (most data analysis algorithms are acceptable -including regression).
●	An interactive “dashboard.” 
○	The application must be usable for solving the proposed problem. Any method enabling the user to interact is acceptable, including the command line. A GUI is not required.
●	A “user-friendly” interface. 
○	Following the “user guide” of part D, the evaluator can successfully run your application as described on their machine.


data_analysis_window

In [196]:
# data_analysis_window --- data analysis window

#TITLE
title = 'Data Analysis (Descriptive Methods)'

#Total Recipes
total_recipes = str(len(recipe_dict))

#Total Ingredients
total_ingredients = str(len(ingredient_dict))

#Mean Ingredients/Recipes
ingredients_per_recipe = 0

#Mean Recipes/Ingredient
recipes_per_ingredient = 0

data_html = widgets.HTML('<style> ' +
                        'table { ' +
                        'font-family: arial, sans-serif;' +
                        ' border-collapse: collapse;' +
                        ' width: 80%;}' +
                        'td, th {' +
                        'border: 1px solid #dddddd;' +
                        ' text-align: left;' +
                        ' padding: 8px;' +
                        '}' +
                        ' tr:nth-child(even) {' +
                        ' background-color: #dddddd;' +
                        ' }' +
                        '</style>' + 
                        '<h1>' + title + '</h1><br><br>' +
                        '<table>' +
                        '<tr><td>Total Recipes: </td><td>' + total_recipes + '</td></tr>' +
                        '<tr><td>Total Ingredients: </td><td>' + total_ingredients + '</td></tr>' +
                        '<tr><td>Mean Ingredients / Recipe: </td><td>' + str(ingredients_per_recipe) + '</td></tr>' +
                        '<tr><td>Mean Recipes / Ingredient: </td><td>' + str(recipes_per_ingredient) + '</td></tr>' +
                        '</table>')
    

data_analysis_window = data_html

recipe_window

In [218]:
# recipe_window --- ingredient exploration window

# list of all recipes for a selected ingredient. Runs off of dict(recipes).
# drop box of ingredients
sorted_ingredients = [x.title() for x in sorted(ingredient_dict.keys())]
ingredient_selector = widgets.Dropdown(options=sorted_ingredients)

# list of all recipes in that ingredient
recipe_list_box = widgets.Select(rows=20)

# list of all ingredients in selected recipe
list_title = widgets.HTML('Ingredients for selected drink:')
drink_ingredients_list = widgets.Select(rows=20)

def update_recipe_list(*args):
    recipe_list_box.options = ingredient_dict[ingredient_selector.value.lower()]
def update_ingredients_list(*args):
    drink_ingredients_list.options = recipe_dict[recipe_list_box.value]

# update recipe list when an ingredient is chosen
ingredient_selector.observe(update_recipe_list, names='value')
# update ingredient list when a recipe is chosen
recipe_list_box.observe(update_ingredients_list, names='value')

# instructions for window
instructions = widgets.HTML('<h3>Choose an ingredient to see which recipes it is used in.<br>' +
                           'Choose a recipe to see which ingredients it uses.')

# arrange and display widgets
left_side = widgets.VBox([ingredient_selector, recipe_list_box])
right_side = widgets.VBox([list_title, drink_ingredients_list])
bottom_pane = widgets.HBox([left_side, right_side])
recipe_window = widgets.VBox([instructions, bottom_pane])

bar_graph_window

In [219]:
# bar_graph_window --- bar graph of recipes / ingredient
bar_graph_out = widgets.Output()
with bar_graph_out:
    graph_recipes_per_ingredient.show()
bar_graph_window = widgets.VBox([bar_graph_out])

word_cloud_window

In [220]:
# word_cloud_window --- word cloud of ingredient frequency
word_cloud_window = widgets.HTML(wordcloud_ingredients_html)

data_analysis_view

In [222]:
# assemble data analysis view and display

top_window = widgets.HBox([data_analysis_window, recipe_window])
data_analysis_view = widgets.VBox([top_window, bar_graph_window, word_cloud_window], layout=widgets.Layout(padding='50px'))
data_analysis_view

## VIEW

In [96]:
heat_map = plt.imshow(df_sims, cmap='hot', interpolation='nearest')
out1 = widgets.Output()
with out1:
    plt.show()
    #  display(heat_map.figure)

#basic list box layout
list_box_layout = widgets.Layout(height='400px')

top_bar = widgets.HTML(
    value="<center><h1>Barbuilder</h1></n><h3>Drink Ingredient Recommendation System</h3>",
)

#set up ingredients window
#search_box
search_label = widgets.Label(value='Search:')
search_box = widgets.Text(disabled=False,)

#ingredient list
ingredient_list = widgets.Select(options=df_ingredients.index, layout=list_box_layout)

#instruction text
ingredient_instructions = widgets.Label(value='Click ingredients to add to your list of what you own.')

#assemble ingredient window
ingredient_window = widgets.VBox([search_label, search_box, ingredient_list, ingredient_instructions])


#set up customer list window
#customer window hbox
customer_list_header = widgets.Label(value='Customer\'s Ingredients')
recipe_count_header = widgets.Label(value='Total Recipes Used In:')
customer_header_hbox = widgets.HBox([customer_list_header, recipe_count_header])
hidden_box = widgets.Text(layout=widgets.Layout(visibility='hidden'))
customer_list = widgets.Select(layout=list_box_layout)
customer_list_recipe_count = widgets.Select(layout=widgets.Layout(height='400px', width='30px'))
customer_list_hbox = widgets.HBox([customer_list, customer_list_recipe_count])

#instruction text
customer_instructions = widgets.Text(description='Ingredients: ', value='0', disabled=True, layout=widgets.Layout(width='50%'))

#assemble customer list window
customer_list_window = widgets.VBox([customer_header_hbox, hidden_box, customer_list_hbox, customer_instructions])


#set up recommendation window
#recommendation window hbox

#instruction text

#assemble recommendation window
recommendation_window = widgets.VBox([])

#assemble windows into main window
main_window = widgets.HBox([ingredient_window, customer_list_window, recommendation_window])

#set up anything that goes along the bottom of the UI
bottom_bar = widgets.Label(
    value='Instructions...',
)

#roll it all into a GUI
gui = widgets.VBox([top_bar, main_window, bottom_bar])

#load gui
gui

In [64]:
items_layout = widgets.Layout( width='auto')     # override the default width of the button to 'auto' to let the button grow

box_layout = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='solid',
                    width='50%')

words = ['correct', 'horse', 'battery', 'staple']
items = [widgets.Button(description=word, layout=items_layout, button_style='danger') for word in words]
box = widgets.Box(children=items, layout=box_layout)
box2 = widgets.Box(children=[out1], layout=box_layout)
gui = widgets.VBox([box, box2])
gui